In [1]:
from mycode.test_20_tables import fetch_data_from_sqlite, Metadata as sdvMetadata
from sdgx.data_connectors.csv_connector import CsvConnector
from sdgx.models.ml.single_table.ctgan import CTGANSynthesizerModel
from sdgx.synthesizer import Synthesizer
import time
def writetime():
    st = time.time()
    with open("time.log", "a+") as f:
        f.write(str(st) + "\n")

构造数据

In [2]:
metadata, tables = fetch_data_from_sqlite(path='./mycode/data_sqlite.db')
metadata = sdvMetadata(metadata)
metadata.get_tables()
import pandas as pd
result_table = tables["BookLoan"]
x_table = ["Book", "Library", "Student", "Enrollment", "Submission"]
x_key = ['book_id', "library_id", "student_id", "student_id","student_id"]
x_how = ['inner' for i in range(5)]
print(f"{len(result_table)=}")
for i, x_table_name in enumerate(x_table):
    result_table = pd.merge(result_table, tables[x_table_name], on=x_key[i], how=x_how[i])
    print(f"{len(result_table)}", end=" ")
print(len(result_table.columns))
#result_table

len(result_table)=10000
10000 10000 10000 9999 10798 23


In [3]:
result_table = result_table.drop(columns=["title", "author", "name_x", "semester", "location", "name_y", "email"])

In [4]:
result_table = result_table.drop(columns=["isbn"])

In [5]:
# result_table = result_table.drop(columns=["grade_x", "grade_y"])

In [6]:
# result_table.head()

In [7]:
# result_table = result_table[:100]
result_table

,loan_id,book_id,student_id,loan_date,return_date,library_id,date_of_birth,major,enrollment_id,course_id,grade_x,submission_id,assignment_id,submission_date,grade_y
0,1,1,5068,2024-07-19,2025-07-30,1,2003-01-27,1262,5758,9618,C,2462,9937,2022-10-18 02:02:31,B
1,1,1,5068,2024-07-19,2025-07-30,1,2003-01-27,1262,5758,9618,C,3984,4403,2022-11-15 20:13:23,I
2,1,1,5068,2024-07-19,2025-07-30,1,2003-01-27,1262,5758,9618,C,7851,6252,2022-07-31 08:44:12,A
3,1,1,5068,2024-07-19,2025-07-30,1,2003-01-27,1262,9382,5777,A,2462,9937,2022-10-18 02:02:31,B
4,1,1,5068,2024-07-19,2025-07-30,1,2003-01-27,1262,9382,5777,A,3984,4403,2022-11-15 20:13:23,I
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10793,9998,9998,3486,2024-01-14,2024-12-09,9998,2000-08-21,8143,3450,6477,F,8054,1227,2022-09-20 01:25:41,C
10794,9998,9998,3486,2024-01-14,2024-12-09,9998,2000-08-21,8143,3757,4480,None,578,6739,2022-11-01 16:39:13,A
10795,9998,9998,3486,2024-01-14,2024-12-09,9998,2000-08-21,8143,3757,4480,None,8054,1227,2022-09-20 01:25:41,C
10796,9999,9999,2281,2023-12-28,2025-08-23,9999,1996-11-30,1005,9064,7483,B,9247,1546,2022-08-05 15:48:08,B


检查metadata

In [8]:
from sdgx.data_loader import DataLoader
from sdgx.data_models.metadata import Metadata

result_table.to_csv("test_10k_noncat.csv", index=False)
dataset_csv = "test_10k_noncat.csv"
data_connector = CsvConnector(path=dataset_csv)
data_loader = DataLoader(data_connector)
loan_metadata = Metadata.from_dataloader(data_loader)

loan_metadata.discrete_columns

2024-10-09 14:25:23.844 | INFO     | sdgx.data_models.metadata:from_dataloader:294 - Inspecting metadata...
2024-10-09 14:25:23.866 | DEBUG    | sdgx.manager:register:99 - Register for new model: subsetrelationshipinspector
2024-10-09 14:25:23.867 | DEBUG    | sdgx.manager:register:99 - Register for new model: emailinspector
2024-10-09 14:25:23.868 | DEBUG    | sdgx.manager:register:99 - Register for new model: chinamainlandidinspector
2024-10-09 14:25:23.869 | DEBUG    | sdgx.manager:register:99 - Register for new model: chinamainlandmobilephoneinspector
2024-10-09 14:25:23.869 | DEBUG    | sdgx.manager:register:99 - Register for new model: chinamainlandpostcode
2024-10-09 14:25:23.870 | DEBUG    | sdgx.manager:register:99 - Register for new model: chinamainlandunifiedsocialcreditcode
2024-10-09 14:25:23.870 | DEBUG    | sdgx.manager:register:99 - Register for new model: chinamainlandaddressinspector
2024-10-09 14:25:23.871 | DEBUG    | sdgx.manager:register:99 - Register for new mode

{'date_of_birth',
 'grade_x',
 'grade_y',
 'loan_date',
 'return_date',
 'submission_date'}

In [9]:
loan_metadata.datetime_columns

{'date_of_birth', 'loan_date', 'return_date', 'submission_date'}

In [10]:

loan_metadata.column_list.__len__()

15

In [11]:
loan_metadata.datetime_format = {
    key: "%Y-%m-%d" for key in loan_metadata.datetime_columns if key != "submission_date"
}
loan_metadata.datetime_format["submission_date"] = "%Y-%m-%d %H:%M:%S"
loan_metadata.datetime_format

{'date_of_birth': '%Y-%m-%d',
 'return_date': '%Y-%m-%d',
 'loan_date': '%Y-%m-%d',
 'submission_date': '%Y-%m-%d %H:%M:%S'}

In [12]:
loan_metadata.column_encoder = {
    "grade_x": "label",
    "grade_y": "label"
}

In [13]:
loan_metadata.discrete_columns = set([])
set([
    key for key in loan_metadata.discrete_columns if key not in loan_metadata.datetime_columns
])
loan_metadata.label_columns.add("grade_x")
loan_metadata.label_columns.add("grade_y")
print(loan_metadata.label_columns)
print(loan_metadata.int_columns)

{'grade_x', 'grade_y'}
{'enrollment_id', 'student_id', 'submission_id', 'library_id', 'assignment_id', 'course_id', 'loan_id', 'major', 'book_id'}


训练

In [14]:


ctgan = CTGANSynthesizerModel(
        epochs=1,
        batch_size=500,
        # device="cpu"
    )
# Initialize synthesizer, use CTGAN model
synthesizer = Synthesizer(
    metadata=loan_metadata,
    model=ctgan,  # For quick demo
    data_connector=data_connector,
)


writetime()
synthesizer.fit()
writetime()
#synthesizer.save("test_model_10k_id10")

2024-10-09 14:25:27.434 | DEBUG    | sdgx.manager:register:99 - Register for new model: outliertransformer
2024-10-09 14:25:27.434 | DEBUG    | sdgx.manager:register:99 - Register for new model: numericvaluetransformer
2024-10-09 14:25:27.435 | DEBUG    | sdgx.manager:register:99 - Register for new model: nonvaluetransformer
2024-10-09 14:25:27.435 | DEBUG    | sdgx.manager:register:99 - Register for new model: emptytransformer
2024-10-09 14:25:27.436 | DEBUG    | sdgx.manager:register:99 - Register for new model: constvaluetransformer
2024-10-09 14:25:27.437 | DEBUG    | sdgx.manager:register:99 - Register for new model: columnordertransformer
2024-10-09 14:25:27.437 | DEBUG    | sdgx.manager:register:99 - Register for new model: emailgenerator
2024-10-09 14:25:27.437 | DEBUG    | sdgx.manager:register:99 - Register for new model: chnpiigenerator
2024-10-09 14:25:27.438 | DEBUG    | sdgx.manager:register:99 - Register for new model: intvalueformatter
2024-10-09 14:25:27.438 | DEBUG   

{'primary_keys': set(), 'column_list': ['loan_id', 'book_id', 'student_id', 'loan_date', 'return_date', 'library_id', 'date_of_birth', 'major', 'enrollment_id', 'course_id', 'grade_x', 'submission_id', 'assignment_id', 'submission_date', 'grade_y'], 'column_inspect_level': {'id_columns': 20, 'empty_columns': 90, 'english_name_columns': 40, 'chinese_name_columns': 40, 'china_mainland_address_columns': 30, 'china_mainland_mobile_phone_columns': 30, 'datetime_columns': 20, 'email_columns': 30, 'discrete_columns': 10, 'const_columns': 80, 'chinese_company_name_columns': 40, 'china_mainland_postcode_columns': 20, 'china_mainland_id_columns': 30, 'unified_social_credit_code_columns': 30, 'int_columns': 10, 'float_columns': 10, 'bool_columns': 10}, 'pii_columns': set(), 'id_columns': set(), 'int_columns': {'enrollment_id', 'student_id', 'library_id', 'submission_id', 'assignment_id', 'course_id', 'loan_id', 'major', 'book_id'}, 'float_columns': set(), 'bool_columns': set(), 'discrete_columns'

2024-10-09 14:25:27.831 | INFO     | sdgx.data_processors.formatters.datetime:convert:105 - Converting data using DatetimeFormatter... Finished.
2024-10-09 14:25:27.832 | INFO     | sdgx.data_processors.transformers.const:convert:69 - Converting data using ConstValueTransformer...
2024-10-09 14:25:27.832 | INFO     | sdgx.data_processors.transformers.const:convert:77 - Converting data using ConstValueTransformer... Finished.
2024-10-09 14:25:27.833 | INFO     | sdgx.data_processors.transformers.empty:convert:72 - Converting data using EmptyTransformer...
2024-10-09 14:25:27.833 | INFO     | sdgx.data_processors.transformers.empty:convert:76 - Converting data using EmptyTransformer... Finished (No action).
2024-10-09 14:25:27.834 | INFO     | sdgx.data_processors.transformers.column_order:convert:51 - Converting data using ColumnOrderTransformer...
2024-10-09 14:25:27.834 | INFO     | sdgx.data_processors.transformers.column_order:convert:52 - Converting data using ColumnOrderTransforme

  0%|          | 0/21 [00:00<?, ?it/s]

2024-10-09 14:25:42.465 | INFO     | sdgx.models.ml.single_table.ctgan:_fit:382 - Epoch 1, Loss G: -0.4888, Loss D: -0.7513, Time:  1.2506
2024-10-09 14:25:42.467 | INFO     | sdgx.models.ml.single_table.ctgan:fit:225 - CTGAN training finished.
2024-10-09 14:25:42.467 | INFO     | sdgx.synthesizer:fit:333 - Model fit... Finished


In [15]:
data_res = synthesizer.sample(500)

2024-10-09 14:25:42.476 | INFO     | sdgx.synthesizer:sample:357 - Sampling...


  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

2024-10-09 14:25:42.513 | INFO     | sdgx.data_processors.transformers.nan:reverse_convert:155 - Data reverse-converted by NonValueTransformer.
2024-10-09 14:25:42.514 | INFO     | sdgx.data_processors.transformers.outlier:reverse_convert:120 - Data reverse-converted by OutlierTransformer (No Action).
2024-10-09 14:25:42.515 | INFO     | sdgx.data_processors.formatters.int:reverse_convert:64 - Data reverse-converted by IntValueFormatter.
2024-10-09 14:25:42.516 | INFO     | sdgx.data_processors.formatters.datetime:reverse_convert:165 - Data reverse-converting by DatetimeFormatter...
2024-10-09 14:25:42.516 | INFO     | sdgx.data_processors.formatters.datetime:reverse_convert:167 - parameters : ['date_of_birth', 'return_date', 'submission_date', 'loan_date'], {'date_of_birth': '%Y-%m-%d', 'return_date': '%Y-%m-%d', 'loan_date': '%Y-%m-%d', 'submission_date': '%Y-%m-%d %H:%M:%S'}
2024-10-09 14:25:42.525 | INFO     | sdgx.data_processors.formatters.datetime:reverse_convert:173 - Data reve

In [16]:
data_res

,loan_id,book_id,student_id,loan_date,return_date,library_id,date_of_birth,major,enrollment_id,course_id,grade_x,submission_id,assignment_id,submission_date,grade_y
0,5997,7623,10124,2024-03-25,2025-01-13,10736,2007-04-06,4911,1283,3808,B,11288,1401,2022-04-20 03:29:04,B
1,5028,7441,7819,2024-02-12,2025-07-25,900,2002-12-05,3200,2186,9761,A,6951,8612,2022-03-02 05:12:55,B
2,8603,1022,1743,2024-02-04,2025-08-20,10118,1999-08-26,5328,1819,10414,A,1387,6165,2022-09-25 18:33:55,A
3,10797,2081,7421,2024-01-04,2025-09-17,9031,2002-05-11,4394,248,6415,A,4970,5316,2022-05-18 13:28:18,B
4,4251,1664,3634,2023-11-04,2025-05-18,9356,2007-10-23,8334,8509,7713,A,7181,8391,2022-05-13 04:37:21,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,10730,3157,5983,2024-06-24,2025-05-31,2084,2001-07-30,418,3154,4670,A,3345,9865,2022-06-29 08:48:55,A
496,3514,6872,3262,2024-03-06,2025-01-20,10097,2001-08-29,1938,3657,9650,B,5847,7980,2021-12-15 10:11:33,A
497,10357,10258,7308,2023-10-27,2025-08-27,2374,1999-03-03,1274,9728,10563,A,1702,8976,2022-09-10 13:04:48,A
498,8341,9193,2984,2024-07-16,2025-05-12,6275,2007-07-25,5707,2690,3516,A,4656,4910,2022-08-14 07:15:19,A
